# DBI strategies comparison

This notebook is a comparison of the so-far developed diagonalization strategies for DBI, including the canonical, Pauli-Z, and magnetic field strategies. On top of these, we also show case the use of invariant DBI generators such as 'BHMM'.

In [ ]:

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt
from qibo import symbols
from qibo import hamiltonians, set_backend
from qibo.hamiltonians import Hamiltonian, SymbolicHamiltonian
from qibo.quantum_info import random_hermitian
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration, DoubleBracketCostFunction
from qibo.models.dbi.utils import generate_Z_operators, generate_pauli_operator_dict, decompose_into_pauli_basis, ParameterizationTypes
from qibo.models.dbi.utils_dbr_strategies import select_best_dbr_generator, gradient_descent, polynomial_step

In [ ]:
def visualize_matrix(matrix, title=""):
    """Visualize hamiltonian in a heatmap form."""
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_title(title)
    try:
        im = ax.imshow(np.absolute(matrix), cmap="inferno")
    except TypeError:
        im = ax.imshow(np.absolute(matrix.get()), cmap="inferno")
    fig.colorbar(im, ax=ax)


## Test on random Hamiltonian


In [ ]:
# backend
set_backend("qibojit", platform="numba")
# initialize dbi object
nqubits = 5
h0 = random_hermitian(2**nqubits, seed=2)
dbi = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0))
cost = DoubleBracketCostFunction.off_diagonal_norm
print("Initial loss", dbi.least_squares(d=dbi.diagonal_h_matrix))
visualize_matrix(dbi.h.matrix, title=f'Random hamiltonian with L={nqubits}')

In [ ]:
# iterations steps
NSTEPS = 5
# choose polynomial scheduling
scheduling = DoubleBracketScheduling.simulated_annealing

### Canonical

In [ ]:
# initialize DBI class for the canonical case
dbi_canonical = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), mode=DoubleBracketGeneratorType.canonical, scheduling=scheduling, cost=cost)

In [ ]:
# Canonical
loss_history_canonical = [dbi_canonical.off_diagonal_norm]
steps_canonical_plot = [0]
for s in range(NSTEPS):
    # same settings as iteration from list
    d = dbi.diagonal_h_matrix
    step = dbi_canonical.choose_step(d=d)
    dbi_canonical(step=step)
    print(f"New optimized step at iteration {s+1}/{NSTEPS}: {step}, loss {dbi_canonical.off_diagonal_norm}")
    loss_history_canonical.append(dbi_canonical.off_diagonal_norm)
    steps_canonical_plot.append(steps_canonical_plot[-1]+step)

### Pauli-Z

In [ ]:
# initialize DBI class for the Pauli-Z strategy
dbi_pauli = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), mode=DoubleBracketGeneratorType.single_commutator, scheduling=scheduling, cost=cost)

In [ ]:
generate_local_Z = generate_Z_operators(nqubits)
Z_ops = list(generate_local_Z.values())
Z_names = list(generate_local_Z.keys())
Z_optimal = []
# add in initial values for plotting
loss_history_pauli = [dbi_pauli.off_diagonal_norm]
steps_pauli_plot = [0]
scheduling = DoubleBracketScheduling.simulated_annealing
for _ in range(NSTEPS):
    dbi_pauli, idx, step, flip_sign = select_best_dbr_generator(dbi_pauli, Z_ops, scheduling=scheduling, compare_canonical=False)
    d = Z_ops[idx]
    loss_history_pauli.append(dbi_pauli.off_diagonal_norm)
    steps_pauli_plot.append(steps_pauli_plot[-1]+step)
    if flip_sign < 0:
        Z_optimal.append('-' + Z_names[idx])
    else:
        Z_optimal.append(Z_names[idx])
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {step} with operator {Z_optimal[-1]}, loss {dbi_pauli.off_diagonal_norm}")

### Magnetic field

In [ ]:
# initialize DBI class for the canonical case
dbi_gradient = DoubleBracketIteration(Hamiltonian(nqubits=nqubits, matrix=h0), mode=DoubleBracketGeneratorType.single_commutator, scheduling=scheduling, cost=cost)

In [ ]:
pauli_operator_dict = generate_pauli_operator_dict(nqubits=nqubits, parameterization_order=2)
d_coef = decompose_into_pauli_basis(dbi.h.matrix, list(pauli_operator_dict.values()))
d = sum([d_coef[i] * list(pauli_operator_dict.values())[i] for i in range(nqubits)])

In [ ]:
def s_hist_to_plot(s_hist):
    # convert list of step durations taken to plotable
    s_plot = [0] * len(s_hist)
    for i in range(len(s_hist)):
        if i != 0:
            s_plot[i] = s_plot[i-1] + s_hist[i]
    return s_plot

In [ ]:
loss_history_gradient, d_params_hist, s_hist = gradient_descent(dbi_gradient, NSTEPS, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)
steps_gradient_plot = s_hist_to_plot(s_hist)

In [ ]:
plt.title(str(nqubits) + ' random Hamiltonian diagonalization')
plt.plot(loss_history_canonical, label='canonical')
plt.plot(loss_history_pauli, label='Pauli-Z')
plt.plot(loss_history_gradient, label='gradient')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

In [ ]:
plt.title(str(nqubits) + ' random Hamiltonian diagonalization')
plt.plot(steps_canonical_plot, loss_history_canonical, marker='o', label='canonical')
plt.plot(steps_pauli_plot, loss_history_pauli, marker='o', label='Pauli-Z')
plt.plot(steps_gradient_plot,loss_history_gradient, marker='o', label='gradient')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

## Test on TFIM


In [ ]:
# backend
set_backend("qibojit", platform="numba")
# initialize dbi object
# hamiltonian parameters
# define the hamiltonian
nqubits = 5
h = 1
H_TFIM = SymbolicHamiltonian( - h*symbols.Z(nqubits-1), nqubits=nqubits)
# add linear interaction terms
for i in range(nqubits-1):
    H_TFIM -= SymbolicHamiltonian(symbols.X(i)*symbols.X(i+1) + h*symbols.Z(i), nqubits=nqubits)
H_TFIM = H_TFIM.dense

# initialize class
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.canonical)
print("Initial off diagonal norm", dbi.off_diagonal_norm)
visualize_matrix(dbi.h.matrix, title=f'Random hamiltonian with L={nqubits}')

In [ ]:
# iterations steps
NSTEPS = 5
# choose polynomial scheduling
scheduling = DoubleBracketScheduling.simulated_annealing

### Canonical

In [ ]:
# initialize DBI class for the canonical case
dbi_canonical = DoubleBracketIteration(deepcopy(H_TFIM), mode=DoubleBracketGeneratorType.canonical, scheduling=scheduling)

In [ ]:
# Canonical
off_diagonal_norm_history_canonical = [dbi_canonical.off_diagonal_norm]
steps_canonical_plot = [0]
for s in range(NSTEPS):
    # same settings as iteration from list
    step = dbi_canonical.choose_step(d=dbi.diagonal_h_matrix)
    dbi_canonical(step=step)
    print(f"New optimized step at iteration {s+1}/{NSTEPS}: {step}, loss {dbi_canonical.off_diagonal_norm}")
    off_diagonal_norm_history_canonical.append(dbi_canonical.off_diagonal_norm)
    steps_canonical_plot.append(steps_canonical_plot[-1]+step)

### Pauli-Z

In [ ]:
# initialize DBI class for the Pauli-Z strategy
dbi_pauli = DoubleBracketIteration(deepcopy(H_TFIM), mode=DoubleBracketGeneratorType.single_commutator, scheduling=scheduling)

In [ ]:
generate_local_Z = generate_Z_operators(nqubits)
Z_ops = list(generate_local_Z.values())
Z_names = list(generate_local_Z.keys())
Z_optimal = []
# add in initial values for plotting
off_diagonal_norm_history_pauli = [dbi_pauli.off_diagonal_norm]
steps_pauli_plot = [0]
scheduling = DoubleBracketScheduling.simulated_annealing
for _ in range(NSTEPS):
    dbi_pauli, idx, step, flip_sign = select_best_dbr_generator(dbi_pauli, Z_ops, scheduling=scheduling, compare_canonical=False)
    off_diagonal_norm_history_pauli.append(dbi_pauli.off_diagonal_norm)
    steps_pauli_plot.append(steps_pauli_plot[-1]+step)
    if flip_sign < 0:
        Z_optimal.append('-' + Z_names[idx])
    else:
        Z_optimal.append(Z_names[idx])
    print(f"New optimized step at iteration {_+1}/{NSTEPS}: {step} with operator {Z_optimal[-1]}, loss {dbi_pauli.off_diagonal_norm}")

### Magnetic field

In [ ]:
# initialize DBI class for the canonical case
dbi_gradient = DoubleBracketIteration(deepcopy(H_TFIM), mode=DoubleBracketGeneratorType.single_commutator, scheduling=scheduling)

In [ ]:
pauli_operator_dict = generate_pauli_operator_dict(nqubits=nqubits, parameterization_order=2)
d_coef = decompose_into_pauli_basis(dbi.h.matrix, list(pauli_operator_dict.values()))
d = sum([d_coef[i] * list(pauli_operator_dict.values())[i] for i in range(nqubits)])

In [ ]:
initial_s = polynomial_step(dbi_object=dbi, d=d, n=4)
print(initial_s)

In [ ]:
off_diagonal_norm_history_gradient, d_params_hist, s_hist = gradient_descent(dbi_gradient, NSTEPS, d_coef, parameterization=ParameterizationTypes.pauli, pauli_operator_dict=pauli_operator_dict)
steps_gradient_plot = s_hist_to_plot(s_hist)

In [ ]:
plt.title(str(nqubits) + ' random Hamiltonian diagonalization')
plt.plot(off_diagonal_norm_history_canonical, label='canonical', marker='o')
plt.plot(off_diagonal_norm_history_pauli, label='Pauli-Z', marker='o')
plt.plot(off_diagonal_norm_history_gradient, label='gradient', marker='o')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')

In [ ]:
plt.title(str(nqubits) + ' random Hamiltonian diagonalization')
plt.plot(steps_canonical_plot, off_diagonal_norm_history_canonical, marker='o', label='canonical')
plt.plot(steps_pauli_plot, off_diagonal_norm_history_pauli, marker='o', label='Pauli-Z')
plt.plot(steps_gradient_plot,off_diagonal_norm_history_gradient, marker='o', label='gradient')
plt.legend()
plt.xlabel('Duration')
plt.ylabel(r'$|| \sigma(e^{sW}He^{-sW}) || $')